In [1]:
# -*- coding:utf-8 -*-
import json
import requests
import csv
import re
from lxml import etree
from datetime import date,datetime
import pandas as pd
import math
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

In [49]:
def save_interest_rate(data):
    
    # 1. 创建文件对象
    f = open('CB_interest_rate.test.csv', 'w', encoding='utf-8', newline='')
    # 2. 基于文件对象构建 csv写入对象
    csv_writer = csv.writer(f)
    # 3. 构建列表头
    csv_writer.writerow(["id","iry1", "iry2", "iry3", "iry4", "iry5", "iry6"])
    
    count = 0
    for item in data:
        count +=1
        if (count %10 == 0):
            print("processing "+str(count)+" items")
        try:
            interest = get_interest_rate(item['id'])
            interest = [item['id']]+interest
            csv_writer.writerow(interest)
        except Exception as e:
            print("Oops!", e.__class__, "occurred.")
            print("The error happed at "+item['id'])
    # 5. 关闭文件
    print("Finished processing "+str(count)+" items")
    f.close()


In [95]:
def get_interest_rate(id):
    years = 6
    detials_url = 'https://www.jisilu.cn/data/convert_bond_detail/'+id
    detials_response = requests.get(detials_url)
    html = detials_response.content.decode("utf-8")
    html = etree.HTML(html)
    interest_text = html.xpath('.//td[@id="cpn_desc"]/text()')
    if(not interest_text[0].find('%')):
        print(" not in interest test %s %s"%(id,interest_text))
        pattern = re.compile(r'(?<!\d|\.)\d+(?:\.\d+)?')  #get proper format number
    else:
        pattern = re.compile(r'(?<!\d|\.)\d+(?:\.\d+)?\s?%')  #get proper format number
    interest = pattern.findall(interest_text[0])
    if(len(interest) <= 1):
        print("%s fixed interest rate %s"%(id, interest_text[0]))
        interest = interest*years
    elif(len(interest)<5):
        print("%s number of interest rate between 2-4 %s"%(id, interest_text[0]))
        print("You may need to check manually")
    elif(len(interest)>6):
        print("%s number of interest rate greater than 6 %s"%(id, interest_text[0]))
        print("You may need to check manually")
    interest = [rate.strip("%") for rate in interest]
    return interest

In [48]:
def get_dat():
    # Fill in your details here to be posted to the login form.
    payload = {
        'user_name': 'a39856ed4377d1b92cffb0677febf5b8',
        'password': 'ca5572b5ffe8c33746727709bc7d6c6f'
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
        #"User-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41",
        "Cookie": "kbzw__Session=l313fggdcjpnd7eineqilm6lf4; kbz_newcookie=1; kbzw_r_uname=ijayden; Hm_lvt_164fe01b1433a19b507595a43bf58262=1627073048,1627079966,1627115858,1627117055; Hm_lpvt_164fe01b1433a19b507595a43bf58262=1627118717"
    }
    loginUrl="https://www.jisilu.cn/account/ajax/login_process/"
    r = requests.post(loginUrl,data=payload)

    newUrl ="https://www.jisilu.cn/data/cbnew/cb_list/?___jsl=LST___t=1584777951900"
    #最简单的爬虫请求.也可以加上headers字段，防止部分网址的反爬虫机制
    response = requests.get(newUrl,cookies=r.cookies)
    data = response.content.decode("utf-8")
    dat = json.loads(data)

    # 所有数据
    lst_data = []
    count = 0
    for one in dat['rows']:
        # 每一条数据
        lst_dat = dict()
        # 转债id
        id = one["id"]
        dat_cell = one["cell"]
        # 是否赎回
        is_force_redeem = dat_cell['redeem_flag']
        is_list = dat_cell['price_tips']
        
        if is_list != "待上市":
            
            count += 1
            if (count %100 == 0):
                print("processing "+str(count)+" items")
            
            # 转债名称
            name = dat_cell['bond_nm']
            # 现价
            price = dat_cell['price']
            # stock id
            stock_id = dat_cell['stock_id']
            stock_id = stock_id.replace("sh", "ss")
            stock_id = stock_id[2:]+"."+stock_id[0:2]
            # stock name
            stock_nm = dat_cell['stock_nm']
            # 溢价率
            premium_rt = dat_cell['premium_rt']
            # 评级
            rating_cd = dat_cell['rating_cd']
            
            
            # next put date
            next_put_dt = dat_cell['next_put_dt']
            # put price
            put_price   = dat_cell['put_price']
            # 回售触发价
            put_convert_price = dat_cell['put_convert_price']
            
            
            # maturity date
            maturity_dt = dat_cell['maturity_dt']
            # redeem price
            redeem_price = dat_cell['redeem_price']
            
            # 强赎触发价
            redeem_dt = dat_cell['redeem_dt']
            force_redeem_price = dat_cell['force_redeem_price']
            # 剩余时间
            last_time = dat_cell['year_left']
            pb = dat_cell['pb']


            #lst_dat.append(id)
            lst_dat['id'] = id
            lst_dat['name'] = name
            lst_dat['stock_id'] = stock_id
            lst_dat['stock_name'] = stock_nm
            lst_dat['price'] = float(price)
            lst_dat['convert_value'] = float(dat_cell['convert_value'])
            lst_dat['premium_rt'] = premium_rt
            lst_dat['rating_cd'] = rating_cd
            lst_dat['pb']  = pb
            lst_dat['convert_amt_ratio'] = dat_cell['convert_amt_ratio']
            lst_dat['convert_amt_ratio2'] = dat_cell['convert_amt_ratio2']
            
            
            
            lst_dat['next_put_dt'] = next_put_dt
            lst_dat['put_price'] = put_price
            lst_dat['put_convert_price'] = put_convert_price
            
            lst_dat['maturity_dt'] = maturity_dt
            lst_dat['redeem_price'] = redeem_price
            lst_dat['last_time'] = last_time
            lst_dat['force_redeem'] = is_force_redeem
            
            lst_dat['redeem_dt']  = redeem_dt
            lst_dat['force_redeem_price'] = force_redeem_price
            
  
            lst_data.append(lst_dat)
            
        else:
            continue
    print("Finished processing "+str(count)+" items")
    
    return lst_data

In [6]:
interest = html.xpath('.//td[@id="cpn_desc"]/text()')
interest

['第一年为 0.40%、第二年为 0.60%、第三年为 1.20%、第四年为 2.20%、 第五年为 3.50%、第六年为 4.00%']

In [46]:
cb1 = get_interest_rate('113049')

In [47]:
cb1

['0.2%', '0.4%', '0.6%', '0.8%', '1.5%', '2.0%']

In [27]:
cb2 = get_interest_rate('120003')

['第1年1.0%,第2年1.0%,第3年1.0%,第4年1.0%,第5年1.0%']


In [25]:
cb3 = get_interest_rate('132018')

['第1年0.5%,第2年0.5%,第3年0.5%,第4年0.5%,第5年0.5%']


In [18]:
cb4 = get_interest_rate('128125')

['第一年 0.4%、第二年 0.6%、第三年 1.0%、第四年 1.5%、第五年 2.0%、第 六年 3.0%。到期赎回价为 113 元(含最后一期利息)']


In [32]:
re.findall(r'(?<!\d|\.)\d+(?:\.\d+)%', cb4[0])

['0.4%', '0.6%', '1.0%', '1.5%', '2.0%', '3.0%']

In [26]:
re.findall(r'(?<!\d|\.)\d+(?:\.\d+)%', cb3[0])

['0.5%', '0.5%', '0.5%', '0.5%', '0.5%']

In [28]:
re.findall(r'(?<!\d|\.)\d+(?:\.\d+)%', cb2[0])

['1.0%', '1.0%', '1.0%', '1.0%', '1.0%']

In [30]:
re.findall(r'(?<!\d|\.)\d+(?:\.\d+)%', cb1[0])

['0.2%', '0.4%', '0.6%', '0.8%', '1.5%', '2.0%']

In [96]:
#data = get_dat()
save_interest_rate(data)

120004 only 1 interest rate 券票面利率为1.0%
processing 10 items
132014 only 1 interest rate 每年固定0.9%
processing 20 items
processing 30 items
processing 40 items
processing 50 items
processing 60 items
processing 70 items
processing 80 items
processing 90 items
processing 100 items
processing 110 items
processing 120 items
processing 130 items
processing 140 items
processing 150 items
processing 160 items
127035 odd number of interest rate 第一年 0.6%、第二年 0.8%、第三年 1.5%、第四年 3.0%、第五年 3.8
processing 170 items
processing 180 items
processing 190 items
processing 200 items
processing 210 items
processing 220 items
processing 230 items
processing 240 items
processing 250 items
processing 260 items
processing 270 items
processing 280 items
processing 290 items
processing 300 items
132022 odd number of interest rate 0.10%,0.10%,0.10%,
processing 310 items
processing 320 items
processing 330 items
processing 340 items
processing 350 items
132009 only 1 interest rate 每年固定1%
132015 only 1 interest rate 每年

In [66]:
get_interest_rate('113569')

113569 2-4 第一年为 0.4%、第二年为 0.8%、第三年为 1%、第四年为 1.5%、第五年 为 2%、第六年为 3%


['0.4%', '0.8%', '1.5%']

In [97]:
detials_url = 'https://www.jisilu.cn/data/convert_bond_detail/'+'127035'
detials_response = requests.get(detials_url)
html = detials_response.content.decode("utf-8")
html = etree.HTML(html)
interest_text = html.xpath('.//td[@id="cpn_desc"]/text()')

In [100]:
pattern = re.compile(r'(?<!\d|\.)\d+(?:\.\d+)?\s?%')  #get proper format number
interest = pattern.findall(interest_text[0])
interest

['0.6%', '0.8%', '1.5%', '3.0%']

In [94]:
interest2 = [rate.strip("?\s%") for rate in interest]
interest2

['0.50', '0.70 ', '1.00 ', '1.50', '2.50', '3.00']

In [99]:
pattern = re.compile(r'(?<!\d|\.)\d+(?:\.\d+)?')  #get proper format number
interest2 = pattern.findall(interest_text[0])
interest2

['0.6', '0.8', '1.5', '3.0', '3.8']

In [56]:
# Fill in your details here to be posted to the login form.
payload = {
    'user_name': 'a39856ed4377d1b92cffb0677febf5b8',
    'password': 'ca5572b5ffe8c33746727709bc7d6c6f'
}
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
    #"User-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.106 Safari/537.36 OPR/38.0.2220.41",
    "Cookie": "kbzw__Session=l313fggdcjpnd7eineqilm6lf4; kbz_newcookie=1; kbzw_r_uname=ijayden; Hm_lvt_164fe01b1433a19b507595a43bf58262=1627073048,1627079966,1627115858,1627117055; Hm_lpvt_164fe01b1433a19b507595a43bf58262=1627118717"
}
loginUrl="https://www.jisilu.cn/account/ajax/login_process/"
r = requests.post(loginUrl,data=payload)

newUrl ="https://www.jisilu.cn/data/cbnew/cb_list/?___jsl=LST___t=1584777951900"
#最简单的爬虫请求.也可以加上headers字段，防止部分网址的反爬虫机制
response = requests.get(newUrl,cookies=r.cookies)
data = response.content.decode("utf-8")
dat = json.loads(data)